In [1]:
import os
import sys
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split

import sys
sys.path.append(os.path.abspath(os.path.join('..')))

In [ ]:
import wandb
wandb.init(project="Final-Project-TimeSeries-UTEC", name="RandomForest-temp-spec", config={
    "n_splits": 7
})

# Models

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class MLPModel(nn.Module):
    def __init__(self, input_size=354, num_classes=6):
        super(MLPModel, self).__init__()
        # Hidden layers with dropout decrecientes
        self.fc1 = nn.Linear(input_size, 256)
        self.bn1 = nn.BatchNorm1d(256)
        self.dropout1 = nn.Dropout(0.4)
        
        self.fc2 = nn.Linear(256, 128)
        self.bn2 = nn.BatchNorm1d(128)
        self.dropout2 = nn.Dropout(0.35)
        
        self.fc3 = nn.Linear(128, 64)
        self.bn3 = nn.BatchNorm1d(64)
        self.dropout3 = nn.Dropout(0.3)
        
        # Output layer
        self.output = nn.Linear(64, num_classes)
        
        # weights init
        self._initialize_weights()
        

    def forward(self, x):
        # negative_slope ayuda a mantener el flujo de gradientes
        x = F.selu(self.fc1(x)) 
        x = self.bn1(x)
        x = self.dropout1(x)
        
        x = F.selu(self.fc2(x))
        x = self.bn2(x)
        x = self.dropout2(x)
        
        x = F.selu(self.fc3(x))
        x = self.bn3(x)
        x = self.dropout3(x)
        
        return self.output(x)
    
    def predict_proba(self, x):
        """Método para compatibilidad con sklearn-style"""
        with torch.no_grad():
            logits = self.forward(x)
            return F.softmax(logits, dim=1).cpu().numpy()
    
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.orthogonal_(m.weight, gain=1.0)  # Inicialización ortogonal
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0.01)  # Pequeño bias inicial
    
class LSTMModel(nn.Module): 
    def __init__(self, input_size=354, num_classes=6, hidden_size=128, num_layers=2):
        super(LSTMModel, self).__init__()
        
        # Capa LSTM
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,  # Para que la entrada sea (batch, seq, feature)
            dropout=0.3 if num_classes > 1 else 0,  # Dropout entre capas LSTM
            #bidirectional=True  # Bidireccional para capturar mejor la temporalidad
        )
        
        # Capas de atencion
        self.attention = nn.Sequential(
            nn.Linear(hidden_size, hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, 1),
            nn.Softmax(dim=1)
        )
        
        # Capas fully connected
        self.fc1 = nn.Linear(hidden_size, 64)
        self.bn1 = nn.BatchNorm1d(64)
        self.dropout = nn.Dropout(0.4)
        
        self.fc2 = nn.Linear(64, num_classes)
        
    def forward(self, x):
        # x debe tener forma (batch_size, seq_length, input_size)
        lstm_out, (h_n, c_n) = self.lstm(x)
        
        # Aplicar atención
        attn_weights = self.attention(lstm_out)
        attn_weights = attn_weights.squeeze(2) 
        attn_weights = F.softmax(attn_weights, dim=1)  # Normalizar pesos de atención
        attn_output = torch.bmm(attn_weights.unsqueeze(1), lstm_out).squeeze(1)  # (batch_size, hidden_size)
        
        # Capas fully connected
        x = F.relu(self.fc1(attn_output))
        x = self.bn1(x)
        x = self.dropout(x)
        
        x = self.fc2(x)
        
        return x
        


class CNNModel(nn.Module):
    def __init__(self, input_channels=1, num_classes=6, input_features=354):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv1d(input_channels, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2)
        )
        self.conv2 = nn.Sequential(
            nn.Conv1d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2)
        )
        self.conv3 = nn.Sequential(
            nn.Conv1d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2)
        )
        # Calcula el tamaño de entrada a fc1 automáticamente
        with torch.no_grad():
            dummy = torch.zeros(1, input_channels, input_features)
            dummy = self.conv1(dummy)
            dummy = self.conv2(dummy)
            dummy = self.conv3(dummy)
            self._to_linear = dummy.view(1, -1).shape[1]
        self.fc1 = nn.Linear(self._to_linear, 512)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
        if x.dim() == 2:
            x = x.unsqueeze(1)
        elif x.dim() == 3 and x.size(1) != 1:
            x = x.permute(0, 2, 1)
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# Utils

In [3]:
import wandb
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, balanced_accuracy_score, accuracy_score
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold, cross_validate

from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier

from copy import deepcopy
from sklearn.base import clone as sk_clone

import time

In [ ]:
def train_one_epoch(model, dataloader, criterion, optimizer, device, epoch=None):
    model.train()
    running_loss = 0.0
    for X_batch, y_batch in dataloader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * X_batch.size(0)
    epoch_loss = running_loss / len(dataloader.dataset)
    #if epoch is not None:
    #    wandb.log({"train_loss": epoch_loss, "epoch": epoch})
    return epoch_loss

def evaluate(model, dataloader, device):
    model.eval()
    all_preds = []
    all_probs = []
    all_labels = []
    start_time = time.time()
    
    with torch.no_grad():
        for X_batch, y_batch in dataloader:
            X_batch = X_batch.to(device)
            outputs = model(X_batch)
            probs = F.softmax(outputs, dim=1)
            
            preds = torch.argmax(outputs, dim=1).cpu().numpy()
            all_preds.extend(preds)
            all_probs.extend(probs.cpu().numpy())
            all_labels.extend(y_batch.numpy())
    
    eval_time = time.time() - start_time
    all_labels = np.array(all_labels)
    all_preds = np.array(all_preds)
    all_probs = np.array(all_probs)
    
    # Calcular métricas
    metrics = {
        'f1': f1_score(all_labels, all_preds, average='weighted'),
        'bal_acc': balanced_accuracy_score(all_labels, all_preds),
        'accuracy': accuracy_score(all_labels, all_preds),
        'roc_auc': roc_auc_score(
            label_binarize(all_labels, classes=np.unique(all_labels)),
            all_probs,
            multi_class='ovr'
        ) if len(np.unique(all_labels)) > 2 else roc_auc_score(all_labels, all_probs[:, 1]),
        'eval_time': eval_time
    }
    
    return metrics, all_preds, all_probs, all_labels



def train_and_test_deepmodel(model, X_train, y_train, X_test, y_test, model_name, dataset_name, save_path, epochs):
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    
    # Preparar DataLoaders
    train_ds = TensorDataset(torch.FloatTensor(X_train.values), torch.LongTensor(y_train.values))
    test_ds = TensorDataset(torch.FloatTensor(X_test.values), torch.LongTensor(y_test.values))
    train_loader = DataLoader(train_ds, batch_size=128, shuffle=True)
    test_loader = DataLoader(test_ds, batch_size=256)
    
    # Entrenamiento
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
    criterion = nn.CrossEntropyLoss()
    
    train_start = time.time()
    for epoch in range(epochs):  # 10 épocas por defecto
        train_one_epoch(model, train_loader, criterion, optimizer, device)
    train_time = time.time() - train_start
    
    # Evaluación
    eval_start = time.time()
    metrics, _, _, _ = evaluate(model, test_loader, device)
    eval_time = time.time() - eval_start
    
    # Guardar modelo
    filename = f"{model_name.lower()}_features_{dataset_name.lower()}.pth"
    torch.save(model.state_dict(), os.path.join(save_path, filename))
    
    return {
        'Modelo': model_name,
        'F1-score': metrics['f1'],
        'Balanced accuracy': metrics['bal_acc'],
        'Accuracy': metrics['accuracy'],
        'ROC-AUC': metrics['roc_auc'],
        'Tiempo entrenamiento (s)': train_time,
        'Tiempo predicción (s)': eval_time,
        'Tiempo total (s)': train_time + eval_time,
        'Archivo': filename
    }

In [ ]:
def kfold_mlp(X, y, name_model ,model_class, model_kwargs, epochs=10, batch_size=128, n_splits=7, device='cpu'):
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    # Resultados a recolectar (formato consistente con modelos clásicos)
    results = {
        'Modelo': name_model,
        'F1-score': [],
        'Balanced accuracy': [],
        'Accuracy': [],
        'ROC-AUC': [],
        'Tiempo fold (s)': []
    }
    
    for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
        fold_start_time = time.time()
        
        # Inicializar modelo
        model = model_class(**model_kwargs).to(device)
        optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
        criterion = nn.CrossEntropyLoss()
        
        # Preparar DataLoaders
        train_ds = TensorDataset(torch.FloatTensor(X[train_idx]), torch.LongTensor(y[train_idx]))
        val_ds = TensorDataset(torch.FloatTensor(X[val_idx]), torch.LongTensor(y[val_idx]))
        train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_ds, batch_size=batch_size*2)
        
        # Entrenamiento
        train_time = 0
        for epoch in range(epochs):
            epoch_start = time.time()
            train_loss = train_one_epoch(model, train_loader, criterion, optimizer, device)
            train_time += time.time() - epoch_start
            
        # Evaluación
        eval_start = time.time()
        metrics, _, _, _ = evaluate(model, val_loader, device)
        eval_time = time.time() - eval_start
        
        # Calcular tiempo total del fold (entrenamiento + evaluación)
        fold_time = time.time() - fold_start_time
        
        # Guardar resultados
        results['F1-score'].append(metrics['f1'])
        results['Balanced accuracy'].append(metrics['bal_acc'])
        results['Accuracy'].append(metrics['accuracy'])
        results['ROC-AUC'].append(metrics['roc_auc'])
        results['Tiempo fold (s)'].append(fold_time)
    
    # Crear DataFrame en formato consistente con modelos clásicos
    df_results = pd.DataFrame({
        'Modelo': results['Modelo'],
        'F1-score': np.mean(results['F1-score']),
        'Balanced accuracy': np.mean(results['Balanced accuracy']),
        'Accuracy': np.mean(results['Accuracy']),
        'ROC-AUC': np.mean(results['ROC-AUC']),
        'Tiempo promedio (s)': np.mean(results['Tiempo fold (s)']),
        'Tiempo total (s)': np.sum(results['Tiempo fold (s)'])
    }, index=[0])
    
    return df_results

In [59]:
from datetime import datetime

def setup_wandb(project_name="Final-Project-TimeSeries-UTEC", model_type="traditional"):
    """Configuración inicial de wandb para cualquier tipo de modelo"""
    wandb.init(
        project=project_name,
        config={
            "model_type": model_type,
            "timestamp": datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
        }
    )

def log_results_to_wandb(results_df, dataset_type, evaluation_type):
    """
    Loggear resultados a wandb de forma consistente para todos los modelos
    
    Args:
        results_df: DataFrame con columnas estandarizadas
        dataset_type: "temporales" o "espectrales"
        evaluation_type: "validación_cruzada" o "evaluación_test"
    """
    required_columns = {
        'Modelo', 'F1-score', 'Balanced accuracy', 'Accuracy', 'ROC-AUC',
        'Tiempo promedio (s)', 'Tiempo total (s)'
    }
    
    # Verificar columnas requeridas
    assert required_columns.issubset(results_df.columns), f"Faltan columnas requeridas: {required_columns - set(results_df.columns)}"
    
    # Loggear cada fila del DataFrame
    for _, row in results_df.iterrows():
        log_data = {
            "Modelo": row["Modelo"],
            "Dataset": dataset_type,
            "Tipo": evaluation_type,
            "F1-score": row["F1-score"],
            "Balanced_accuracy": row["Balanced accuracy"],
            "Accuracy": row["Accuracy"],
            "ROC-AUC": row["ROC-AUC"]
        }
        
        # Añadir tiempos según el tipo de evaluación
        if evaluation_type == "validación_cruzada":
            log_data.update({
                "Tiempo_promedio(s)": row["Tiempo promedio (s)"],
                "Tiempo_total(s)": row["Tiempo total (s)"]
            })
        else:  # evaluación_test
            log_data.update({
                "Tiempo_entrenamiento(s)": row["Tiempo entrenamiento (s)"],
                "Tiempo_predicción(s)": row["Tiempo predicción (s)"],
                "Tiempo_total(s)": row["Tiempo total (s)"]
            })
        
        wandb.log(log_data)
    
    # Loggear tabla resumen
    wandb.log({
        f"Resumen_{evaluation_type}_{dataset_type}": wandb.Table(dataframe=results_df)
    })

# Train_mlp

In [36]:
epochs = 10
n_splits = 7

## Kfold

In [6]:
project_root = os.path.dirname(os.getcwd())
data_path = os.path.join(project_root, 'data')
processed_path = data_path + '/processed'

save_models_path = os.path.join(project_root, 'models', 'save_models')
save_results_path = os.path.join(project_root, 'models', 'results')

features_temporal = pd.read_csv(processed_path + '/features_temporales_labelNum_overlap50.csv')
features_espectrales = pd.read_csv(processed_path + '/features_espectrales_labelNum_overlap50.csv')

X_temp = features_temporal.iloc[:, 1:-1] 
y_temp = features_temporal.iloc[:, -1]  
X_spec = features_espectrales.iloc[:, 1:-1] 
y_spec = features_espectrales.iloc[:, -1]  

# Dividir el dataset en entrenamiento y prueba
X_temp_train, X_temp_test, y_temp_train, y_temp_test = train_test_split(X_temp, y_temp, test_size=0.2, random_state=42)
X_spec_train, X_spec_test, y_spec_train, y_spec_test = train_test_split(X_spec, y_spec, test_size=0.2, random_state=42)

In [42]:
df_kfold_temp = kfold_mlp(X_temp_train.values, y_temp_train.values, 
                         "MLP",MLPModel, 
                         {'input_size': X_temp_train.shape[1], 'num_classes': len(np.unique(y_temp_train))},
                         epochs=epochs, device='cuda' if torch.cuda.is_available() else 'cpu')

df_kfold_spec = kfold_mlp(X_spec_train.values, y_temp_train.values, 
                         "MLP",MLPModel, 
                         {'input_size': X_spec_train.shape[1], 'num_classes': len(np.unique(y_spec_train))},
                         epochs=epochs, device='cuda' if torch.cuda.is_available() else 'cpu')


In [43]:
df_kfold_temp

,Modelo,F1-score,Balanced accuracy,Accuracy,ROC-AUC,Tiempo promedio (s),Tiempo total (s)
0,MLP,0.883374,0.820624,0.888606,0.979886,5.835734,40.850136


In [44]:
df_kfold_spec

,Modelo,F1-score,Balanced accuracy,Accuracy,ROC-AUC,Tiempo promedio (s),Tiempo total (s)
0,MLP,0.871793,0.817565,0.880811,0.97639,5.842845,40.899913


In [45]:
csv_cv_temp = os.path.join(save_results_path, 'resultados_cv_temporales.csv')
if os.path.isfile(csv_cv_temp):
    df_cv_temp_old = pd.read_csv(csv_cv_temp)
    df_temp_kfold = pd.concat([df_cv_temp_old, df_kfold_temp], ignore_index=True)
df_temp_kfold.to_csv(csv_cv_temp, index=False)

csv_cv_spec = os.path.join(save_results_path, 'resultados_cv_espectrales.csv')
if os.path.isfile(csv_cv_spec):
    df_cv_spec_old = pd.read_csv(csv_cv_spec)
    df_spec_kfold = pd.concat([df_cv_spec_old, df_kfold_spec], ignore_index=True)
df_spec_kfold.to_csv(csv_cv_spec, index=False)

In [ ]:
log_results_to_wandb(df_kfold_temp, "temporales", "validación_cruzada")
log_results_to_wandb(df_kfold_temp, "espectrales", "validación_cruzada")

# Training-test

In [51]:
dict_test_results_temp = train_and_test_deepmodel(
    model=MLPModel(input_size=X_temp_train.shape[1], num_classes=6),
    X_train=X_temp_train,
    y_train=y_temp_train,
    X_test=X_temp_test,
    y_test=y_temp_test,
    model_name="MLP",
    dataset_name="temporales",
    save_path=save_models_path,
    epochs=epochs
)

dict_test_results_spec = train_and_test_deepmodel(
    model=MLPModel(input_size=X_spec_train.shape[1], num_classes=6),
    X_train=X_spec_train,
    y_train=y_spec_train,
    X_test=X_spec_test,
    y_test=y_spec_test,
    model_name="MLP",
    dataset_name="espectrales",
    save_path=save_models_path,
    epochs=epochs
)

In [54]:
dict_test_results_temp

{'Modelo': 'MLP',
 'F1-score': 0.8888468250636777,
 'Balanced accuracy': 0.832601981309999,
 'Accuracy': 0.8951048951048951,
 'ROC-AUC': 0.9815525079627383,
 'Tiempo entrenamiento (s)': 6.182087421417236,
 'Tiempo predicción (s)': 0.04358339309692383,
 'Tiempo total (s)': 6.22567081451416,
 'Archivo': 'mlp_features_temporales.pth'}

In [58]:
dict_test_results_spec

{'Modelo': 'MLP',
 'F1-score': 0.8672343363334757,
 'Balanced accuracy': 0.8032980908902747,
 'Accuracy': 0.8782051282051282,
 'ROC-AUC': 0.9766339849830432,
 'Tiempo entrenamiento (s)': 9.73764419555664,
 'Tiempo predicción (s)': 0.04715704917907715,
 'Tiempo total (s)': 9.784801244735718,
 'Archivo': 'mlp_features_espectrales.pth'}

In [ ]:

csv_test_temp = os.path.join(save_results_path, "resultados_test_temporales.csv")

# Crear DataFrame con los resultados del MLP
mlp_test_results_temp = {
    "Modelo": "MLP",
    "F1-score": dict_test_results_temp["F1-score"],
    "Balanced accuracy": dict_test_results_temp["Balanced accuracy"],
    "Accuracy": dict_test_results_temp["Accuracy"],
    "ROC-AUC": dict_test_results_temp["ROC-AUC"],
    "Tiempo entrenamiento (s)": dict_test_results_temp["Tiempo entrenamiento (s)"],
    "Tiempo predicción (s)": dict_test_results_temp["Tiempo predicción (s)"],
    "Tiempo total (s)": dict_test_results_temp["Tiempo total (s)"],
    "Archivo": dict_test_results_temp["Archivo"]
}

# Leer archivo existente o crear nuevo DataFrame
if os.path.isfile(csv_test_temp):
    df_test_temp = pd.read_csv(csv_test_temp)
    # Eliminar entrada previa de MLP si existe (para evitar duplicados)
    df_test_temp = df_test_temp[df_test_temp['Modelo'] != 'MLP']
else:
    df_test_temp = pd.DataFrame()

# Concatenar los nuevos resultados
df_test_temp = pd.concat([df_test_temp, pd.DataFrame([mlp_test_results_temp])], ignore_index=True)

# Guardar el archivo actualizado
df_test_temp.to_csv(csv_test_temp, index=False)



In [ ]:
df_test_mlp_temp = pd.DataFrame([mlp_test_results_temp])
log_results_to_wandb(df_test_mlp_temp, "temporales", "evaluación_test")

In [57]:
csv_test_spec = os.path.join(save_results_path, "resultados_test_espectrales.csv")
# Crear DataFrame con los resultados del MLP
mlp_test_results_spec = {
    "Modelo": "MLP",
    "F1-score": dict_test_results_spec["F1-score"],
    "Balanced accuracy": dict_test_results_spec["Balanced accuracy"],
    "Accuracy": dict_test_results_spec["Accuracy"],
    "ROC-AUC": dict_test_results_spec["ROC-AUC"],
    "Tiempo entrenamiento (s)": dict_test_results_spec["Tiempo entrenamiento (s)"],
    "Tiempo predicción (s)": dict_test_results_spec["Tiempo predicción (s)"],
    "Tiempo total (s)": dict_test_results_spec["Tiempo total (s)"],
    "Archivo": dict_test_results_spec["Archivo"]
}

# Leer archivo existente o crear nuevo DataFrame
if os.path.isfile(csv_test_spec):
    df_test_spec = pd.read_csv(csv_test_spec)
    # Eliminar entrada previa de MLP si existe (para evitar duplicados)
    df_test_spec = df_test_spec[df_test_spec['Modelo'] != 'MLP']
else:
    df_test_spec = pd.DataFrame()

# Concatenar los nuevos resultados
df_test_spec = pd.concat([df_test_spec, pd.DataFrame([mlp_test_results_spec])], ignore_index=True)

# Guardar el archivo actualizado
df_test_spec.to_csv(csv_test_spec, index=False)


In [ ]:
df_test_mlp_spec = pd.DataFrame([mlp_test_results_spec])
log_results_to_wandb(df_test_mlp_spec, "espectrales", "evaluación_test")

wandb.finish()